<a href="https://colab.research.google.com/github/Puurun/fastai_code_example/blob/main/musictonote_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Mar 31 13:36:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
from IPython.display import clear_output 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fastai -Uqq
!pip install mido
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/'Colab Notebooks'/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
clear_output()

In [ ]:
from mido import MidiFile
from mido import Message
from fastai.text.all import *

In [ ]:
!kaggle datasets download -d soumikrakshit/classical-music-midi
!unzip /content/classical-music-midi.zip
clear_output()

In [ ]:
mid_files = get_files('/content/chopin', '.mid')

In [ ]:
mid_files

(#48) [Path('/content/chopin/chpn_op10_e05.mid'),Path('/content/chopin/chpn_op25_e12.mid'),Path('/content/chopin/chpn_op7_2.mid'),Path('/content/chopin/chpn_op35_2.mid'),Path('/content/chopin/chpn_op35_4.mid'),Path('/content/chopin/chpn_op27_2.mid'),Path('/content/chopin/chpn-p12.mid'),Path('/content/chopin/chpn-p18.mid'),Path('/content/chopin/chpn-p22.mid'),Path('/content/chopin/chpn-p16.mid')...]

In [ ]:
mids = L(MidiFile(mid) for mid in mid_files)

In [ ]:
piano_right = L(mid.tracks[1] for mid in mids)

In [ ]:
right_nvt = L()

In [ ]:
for mid in piano_right:
    for i, msg in enumerate(mid):
        if not(msg.is_meta) and not(msg.is_cc)():
            right_nvt.append(msg)
                

In [ ]:
right_nvt

(#80657) [Message('program_change', channel=0, program=0, time=0),Message('note_on', channel=0, note=90, velocity=99, time=1),Message('note_on', channel=0, note=94, velocity=76, time=79),Message('note_on', channel=0, note=90, velocity=0, time=1),Message('note_on', channel=0, note=94, velocity=0, time=79),Message('note_on', channel=0, note=85, velocity=69, time=0),Message('note_on', channel=0, note=85, velocity=0, time=80),Message('note_on', channel=0, note=90, velocity=93, time=0),Message('note_on', channel=0, note=90, velocity=0, time=80),Message('note_on', channel=0, note=87, velocity=79, time=0)...]

In [ ]:
right_nvt = L(filter(lambda x: x.type == 'note_on', right_nvt))

In [ ]:
right_nvt = [msg.dict() for msg in right_nvt]

In [ ]:
vocab_note = L(one['note'] for one in right_nvt).unique().sorted()
vocab_velo = L(one['velocity'] for one in right_nvt).unique().sorted()
vocab_time = L(one['time'] for one in right_nvt).unique().sorted()

In [ ]:
dict_note = {}; dict_velo = {}; dict_time = {}

In [ ]:
for i, note in enumerate(vocab_note):
    dict_note[i] = note
for i, velo in enumerate(vocab_velo):
    dict_velo[i] = velo
for i, time in enumerate(vocab_time):
    dict_time[i] = time

In [ ]:
def voc2idx(voc, dict):
    if dict=='note': d=dict_note
    if dict=='velo': d=dict_velo
    if dict=='time': d=dict_time
    for key in d.keys():
        if d[key] == voc:
            return key

In [ ]:
def idx2voc(idx, dict):
    if dict=='note': d=dict_note
    if dict=='velo': d=dict_velo
    if dict=='time': d=dict_time
    return d[idx]

In [ ]:
class numericalize(Transform):
    def encodes(self, x):
        return L([voc2idx(t['note'], 'note'),
                  voc2idx(t['velocity'], 'velo'),
                  voc2idx(t['time'], 'time')] for t in x)
    def decodes(self, x):
        return L({'note': idx2voc(t[0], 'note'),
                  'velo': idx2voc(t[1], 'velo'),
                  'time': idx2voc(t[2], 'time')} for t in x)

In [ ]:
numer = numericalize()

In [ ]:
right_voc = numer(right_nvt)

In [ ]:
numer.decode(right_voc)

(#80610) [{'note': 90, 'velo': 99, 'time': 1},{'note': 94, 'velo': 76, 'time': 79},{'note': 90, 'velo': 0, 'time': 1},{'note': 94, 'velo': 0, 'time': 79},{'note': 85, 'velo': 69, 'time': 0},{'note': 85, 'velo': 0, 'time': 80},{'note': 90, 'velo': 93, 'time': 0},{'note': 90, 'velo': 0, 'time': 80},{'note': 87, 'velo': 79, 'time': 0},{'note': 87, 'velo': 0, 'time': 80}...]

In [ ]:
'''dset_x = [torch.tensor([x['note'], x['velo'], x['time']], dtype=torch.int) for x in right_voc[:-1]]
dset_y = [torch.tensor([y['note'], y['velo'], y['time']], dtype=torch.int) for y in right_voc[1:]]
dset = list(zip(dset_x, dset_y))'''

"dset_x = [torch.tensor([x['note'], x['velo'], x['time']], dtype=torch.int) for x in right_voc[:-1]]\ndset_y = [torch.tensor([y['note'], y['velo'], y['time']], dtype=torch.int) for y in right_voc[1:]]\ndset = list(zip(dset_x, dset_y))"

In [ ]:
#parameter
bs = 64
sl = 6
cut_ratio = 0.7

In [ ]:
right_voc_sl = L(right_voc[i:i+sl] for i in range(0,len(right_voc)))

In [ ]:
right_voc_sl

(#80610) [[[60, 85, 1], [64, 62, 65], [60, 0, 1], [64, 0, 65], [55, 55, 0], [55, 0, 66]],[[64, 62, 65], [60, 0, 1], [64, 0, 65], [55, 55, 0], [55, 0, 66], [60, 79, 0]],[[60, 0, 1], [64, 0, 65], [55, 55, 0], [55, 0, 66], [60, 79, 0], [60, 0, 66]],[[64, 0, 65], [55, 55, 0], [55, 0, 66], [60, 79, 0], [60, 0, 66], [57, 65, 0]],[[55, 55, 0], [55, 0, 66], [60, 79, 0], [60, 0, 66], [57, 65, 0], [57, 0, 66]],[[55, 0, 66], [60, 79, 0], [60, 0, 66], [57, 65, 0], [57, 0, 66], [60, 59, 0]],[[60, 79, 0], [60, 0, 66], [57, 65, 0], [57, 0, 66], [60, 59, 0], [60, 0, 66]],[[60, 0, 66], [57, 65, 0], [57, 0, 66], [60, 59, 0], [60, 0, 66], [55, 78, 0]],[[57, 65, 0], [57, 0, 66], [60, 59, 0], [60, 0, 66], [55, 78, 0], [55, 0, 66]],[[57, 0, 66], [60, 59, 0], [60, 0, 66], [55, 78, 0], [55, 0, 66], [60, 64, 0]]...]

In [ ]:
def get_chunk(dset):
    m = len(dset)//bs
    nd = L()
    for i in range(m): nd += L([dset[i+j*m] for j in range(bs)] )
    return nd

In [ ]:
torch.tensor(get_chunk(right_voc_sl[:-1])).shape

torch.Size([80576, 6, 3])

In [ ]:
dset_x = torch.tensor(get_chunk(right_voc_sl[:-1]))
dset_y = torch.tensor(get_chunk(right_voc_sl[1:]))[:,-1,:]

In [ ]:
dset_x.shape

torch.Size([80576, 6, 3])

In [ ]:
dset = L(zip(dset_x, dset_y))

In [ ]:
cut = int(len(dset)*cut_ratio)

In [ ]:
dls = DataLoaders.from_dsets(dset[:cut],dset[cut:], shuffle=False, drop_last=True, bs=bs)

In [ ]:
first(dls.train)[0].shape

torch.Size([64, 6, 3])

In [ ]:
class MusicModel(Module):
    def __init__(self, input_size, embedding_size, layers):
        self.vs_note, self.vs_velo, self.vs_time = input_size
        self.embedding_size = embedding_size
        self.embed_note = Embedding(self.vs_note, embedding_size)
        self.embed_velo = Embedding(self.vs_velo, embedding_size)
        self.embed_time = Embedding(self.vs_time, embedding_size)
        self.lstm1 = nn.LSTM(embedding_size*3, embedding_size*3, layers, batch_first=True, dropout=0.5)
        self.drop = nn.Dropout(p=0.5)
        self.lstm2 = nn.LSTM(embedding_size*3, embedding_size*3, layers, batch_first=True, dropout=0.5)
        self.linear_note = nn.Linear(embedding_size, self.vs_note)
        self.linear_velo = nn.Linear(embedding_size, self.vs_velo)
        self.linear_time = nn.Linear(embedding_size, self.vs_time)
        self.h = [torch.zeros(layers, bs, embedding_size*3) for _ in range(2)]
    def forward(self, x):
        note, velo, time = x[:,:,0], x[:,:,1], x[:,:,2]
        embed_total = torch.cat([self.embed_note(note), self.embed_velo(velo), self.embed_time(time)], 2)
        embed_total = nn.Dropout(p=0.2)(embed_total)
        res, h = self.lstm1(embed_total, self.h)
        res = self.drop(res)
        res, h = self.lstm2(res, h)
        res = self.drop(res)
        self.h = [h_.detach() for h_ in h]
        final = (self.linear_note(res[:,:,:self.embedding_size].mean(1)), 
                 self.linear_velo(res[:,:,self.embedding_size:self.embedding_size*2].mean(1)),
                 self.linear_time(res[:,:,self.embedding_size*2:].mean(1)))
        return final

In [ ]:
def cross_entropy(input, target):
    target = target.long()
    note_entropy = F.cross_entropy(input[0].view(64,-1), target[:,0].view(64))
    velo_entropy = F.cross_entropy(input[1].view(64,-1), target[:,1].view(64))
    time_entropy = F.cross_entropy(input[2].view(64,-1), target[:,2].view(64))
    return note_entropy + velo_entropy + time_entropy

In [ ]:
input_size = len(vocab_note), len(vocab_velo), len(vocab_time)

In [ ]:
learn = Learner(dls, MusicModel(input_size, 100, 5), loss_func=cross_entropy)

In [ ]:
#learn.fit_one_cycle(3)

In [ ]:
#learn.save('/content/drive/MyDrive/Colab Notebooks/Music_Generation/model1')

In [ ]:
learn.load('/content/drive/MyDrive/Colab Notebooks/Music_Generation/model1')

In [ ]:
class MusicModel2(Module):
     def __init__(self, input_size, embedding_size, layers):
        self.vs_note, self.vs_velo, self.vs_time = input_size
        self.embedding_size = embedding_size
        self.embed_note = Embedding(self.vs_note, embedding_size)
        self.embed_velo = Embedding(self.vs_velo, embedding_size)
        self.embed_time = Embedding(self.vs_time, embedding_size)
        self.lstm1 = nn.LSTM(embedding_size*3, embedding_size*3, layers, batch_first=True, dropout=0.5)
        self.drop = nn.Dropout(p=0.5)
        self.lstm2 = nn.LSTM(embedding_size*3, embedding_size*3, layers, batch_first=True, dropout=0.5)
        self.linear_note = nn.Linear(embedding_size, self.vs_note)
        self.linear_velo = nn.Linear(embedding_size, self.vs_velo)
        self.linear_time = nn.Linear(embedding_size, self.vs_time)
        self.h = [torch.zeros(layers, bs, embedding_size*3) for _ in range(2)]
    def forward(self, x):
        note, velo, time = x[:,:,0], x[:,:,1], x[:,:,2]
        embed_total = torch.cat([self.embed_note(note), self.embed_velo(velo), self.embed_time(time)], 2)
        embed_total = nn.Dropout(p=0.2)(embed_total)
        res, h = self.lstm1(embed_total, self.h)
        res = self.drop(res)
        res, h = self.lstm2(res, h)
        res = self.drop(res)
        self.h = [h_.detach() for h_ in h]
        final = (self.linear_note(res[:,:,:self.embedding_size].mean(1)), 
                 self.linear_velo(res[:,:,self.embedding_size:self.embedding_size*2].mean(1)),
                 self.linear_time(res[:,:,self.embedding_size*2:].mean(1)))
        return final

In [ ]:
def predict(learner, notes, n_words=10):
    idxs = idxs_all = learner.dls.test_dl(notes).to()
    for _ in progress_bar(range(n_words), leave=False):
        preds = learner.get_preds(dl=[(idxs[None],)])

In [ ]:
''''
def predict(self, text, n_words=1, no_unk=True, temperature=1., min_p=None, no_bar=False,
                decoder=decode_spec_tokens, only_last_word=False):
        "Return `text` and the `n_words` that come after"
        self.model.reset()
        idxs = idxs_all = self.dls.test_dl([text]).items[0].to(self.dls.device)
        if no_unk: unk_idx = self.dls.vocab.index(UNK)
        for _ in (range(n_words) if no_bar else progress_bar(range(n_words), leave=False)):
            with self.no_bar(): preds,_ = self.get_preds(dl=[(idxs[None],)])
            res = preds[0][-1]
            if no_unk: res[unk_idx] = 0.
            if min_p is not None:
                if (res >= min_p).float().sum() == 0:
                    warn(f"There is no item with probability >= {min_p}, try a lower value.")
                else: res[res < min_p] = 0.
            if temperature != 1.: res.pow_(1 / temperature)
            idx = torch.multinomial(res, 1).item()
            idxs = idxs_all = torch.cat([idxs_all, idxs.new([idx])])
            if only_last_word: idxs = idxs[-1][None]

        num = self.dls.train_ds.numericalize
        tokens = [num.vocab[i] for i in idxs_all if num.vocab[i] not in [BOS, PAD]]
        sep = self.dls.train_ds.tokenizer.sep
        return sep.join(decoder(tokens))

    @delegates(Learner.get_preds)
    def get_preds(self, concat_dim=1, **kwargs): return super().get_preds(concat_dim=1, **kwargs)
    ''''